In [82]:
import pandas as pd
import numpy as np
import unicodedata
import requests
import bs4
import re
import os

from pathlib import Path
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML



In [83]:
#global constants
MAIN_PAGE_ADDRESS = "https://nccs-data.urban.org/showDD.php?ds=core"
NCCS_SUFFIX = 'https://nccs-data.urban.org/'
INDEX_NAME = 'NCCSMetaIndex.csv'
INDEX_ADDITIONS_NAME = 'NCCSMetaIndexAdditions.csv'
DOCUMENTATION_ROOT = 'data\\documentation\\'
DOCUMENTATION_META = DOCUMENTATION_ROOT + 'raw meta\\'
DOCUMENTATION_SUB = DOCUMENTATION_META + 'subtables\\'

#TODO Missing Dictionaries

FILE_TYPES = {'PC':'Public Charity',
             'PF': 'Private Foundation',
             'others':'Other 501c',
             'UNK':'Unknown'
            }


In [84]:
#acquires the index from the website
def getIndex():
    mainPage = requests.get(MAIN_PAGE_ADDRESS)
    mainSoup = BeautifulSoup(mainPage.text, 'lxml')
    tab = mainSoup.find('table')
    #print(list(tab.children))

    acquiredData =[]
    for r in tab.children:
        if not isinstance(r, bs4.element.NavigableString):
            if r.name=='tr':
                c=r.td
            else:
                c=r
            #for c in r.find_all('td'):
            for a in c.find_all('a'):
                if 'href' in a.attrs and 'align' in c.attrs and c.attrs['align']=='LEFT':
                    b = a.find('b')
                    if b:
                        address = (NCCS_SUFFIX + a.attrs['href'])
                        metaName = b.text
                        fileYear = (re.search('\d{4}', metaName)).group()
                        full = 'Full' in metaName # Full files have multiple years of data
                        legacy = 'Beta' in metaName

                        description = c.text
                        
                        fileType = FILE_TYPES['UNK'] #extracts the file type for the dictionary
                        for typeCode in FILE_TYPES:
                            if ' ' + typeCode in metaName:
                                fileType = FILE_TYPES[typeCode]
                                fileTypeCode = typeCode
                        
                        if fileType == 'Other 501c': #now build out the file name
                            originalFileName = 'coreco.core' + fileYear + 'co'
                            name = "nccs_core_" + fileYear + '_co'
                        else:
                            originalFileName = 'nccs.core' + fileYear + fileTypeCode.lower()
                            name = "nccs_core_" + fileYear +"_" + fileTypeCode.lower()
                        if full:
                            originalFileName +='_full990'
                            name += '_full'
                        if legacy:
                            originalFileName = 'LEGACY'
                            name += '_legacy'
                        acquiredData += [[address, name, originalFileName, fileYear, fileType, full, legacy, description]]                    
    df = pd.DataFrame(acquiredData, columns=['address', 'name', 'originalFileName', 
                                             'year', 'type', 'full', 'legacy', 'description'])
    
    if Path(DOCUMENTATION_ROOT + INDEX_ADDITIONS_NAME).is_file():
        dfAdd = pd.read_csv(DOCUMENTATION_ROOT + INDEX_ADDITIONS_NAME, index_col=0)
        df = pd.concat([df, dfAdd], ignore_index=True)
        
    df.to_csv(DOCUMENTATION_ROOT + INDEX_NAME, index_label='index')
    return df

In [85]:
#Wrapper to get the index
def loadIndex(refreshIndex=True):
    if refreshIndex:
        df = getIndex()
        print("Acquired Metadata")
    else:
        df = pd.read_csv(DOCUMENTATION_ROOT + INDEX_NAME, index_col=0)
    return df

In [86]:
def processSubTable(fileName, fieldName, subTable):
    subResults = []
    hasSubTable = False
    
    for sr in subTable.find_all('tr'):
        subRow = []
        for c in sr.find_all('th'):
            subRow += [c.text]
        for c in sr.find_all('td'):
            subRow += [c.text]
        if len(subRow) > 0:
            subResults += [subRow]
            
    if len(subResults) > 0:
        df = pd.DataFrame(subResults, columns=['value', 'description'])
        outName = DOCUMENTATION_SUB + fileName + "\\" + fileName +' - ' + fieldName + '.csv'
        os.makedirs(os.path.dirname(outName), exist_ok=True) #make the path if it does not already exist
        df.to_csv(outName, index_label='index')
        hasSubTable = True
    return hasSubTable

In [87]:
def processDictionary(fileName, address, saveAsFile=True):

    page = requests.get(address)
    pageSoup = BeautifulSoup(page.text, 'lxml')
    
    #tables = pageSoup.find_all('table')
    mainTable = pageSoup.find_all('table')
    
    results = []
    
    #loop through all table rows
    for r in mainTable[1].children:
        try:
        
            #make sure we are not just looking at text
            if not isinstance(r, bs4.element.NavigableString):
                rChildren = list(r.children)

                #check if we have a traditional column type
                if rChildren[0].name == 'td':               
                    nameAndType = list(rChildren[0].children) #get info on the name and type
                    fieldName = nameAndType[0].text
                    fieldType = nameAndType[2]
                    if len(nameAndType)>3:
                        fieldSize = int(((nameAndType[4])[1:(len(nameAndType[4])-1)]))
                    else:
                        fieldSize = 0
                    longDesc = ""
                    hasSubTable = False
                    for descItem in rChildren[1].children:
                        if isinstance(descItem, bs4.element.NavigableString): #first see if its text
                            longDesc += descItem
                        elif descItem.name == 'b': #if its bold, its probably a title
                            shortDesc = rChildren[1].b.text
                        #parse sub tables in a limited way
                        elif descItem.name == 'table' and len(list(descItem.children)) > 0: 
                            hasSubTable = processSubTable(fileName = fileName, fieldName = fieldName, subTable = descItem)                           
                            for sr in descItem.find_all('tr'):
                                longDesc+= '\n'
                                for sc in sr.children:
                                    if sc.name == 'th' or sc.name == 'td':
                                        if not longDesc[-1] == '\n':
                                            longDesc += ' '
                                        longDesc += sc.text
                        elif descItem.name == 'br': #process linebreaks literally
                            longDesc += '\n'
                        else:
                            longDesc += descItem.text #otherwise, just grab the text
                    results += [[fieldName.lower(), fieldType, fieldSize, hasSubTable, shortDesc, str.strip(longDesc)]]
        except:
            print("WARNING: Failed to parse row. Content: ")
            print(r)   
    df = pd.DataFrame(results, columns=['name', 'type', 'size', 'subTable','shortDesc', 'longDesc'])
    df.to_csv(DOCUMENTATION_META + fileName + '.csv', index_label='index')    
    return results

    
    
    

In [88]:
def navigator(df, update=False):
    for row in df.itertuples():
        fileName = row[2]
        address = row[1]
        if (not update) or (not Path(DOCUMENTATION_META + fileName + '.csv').is_file()):
            processDictionary(fileName = fileName, address = address)
            print("Scraped " + row[2])
    
    return df
        

In [89]:
def getMeta(refreshIndex=True, scrape=True, update=False):
    df = loadIndex(refreshIndex=refreshIndex)
    if scrape:
        navigator(df, update=update)
        print('Scrape successful')
    
    return
    #print(processDictionary(fileName=df['name'][1], address = df['address'][1], saveAsFile=True))
    

In [90]:
getMeta(refreshIndex=True, update=True, scrape=True)

Acquired Metadata
Scrape successful
